<a href="https://colab.research.google.com/github/linztjavier-max/BASC0005-London-Air-Inequality/blob/main/2022_pollution_qm2_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import io
import zipfile
import requests
import pandas as pd

# Make pandas show EVERYTHING
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)


def load_full_pm25_summary(zip_url):
    """
    Downloads LAEI ZIP file and returns the FULL PM2.5 Summary sheet as a DataFrame
    """
    # Download ZIP
    response = requests.get(zip_url)
    response.raise_for_status()

    # Open ZIP
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Find Excel file
        excel_name = [f for f in z.namelist() if f.lower().endswith(".xlsx")][0]

        # Load Excel
        excel_bytes = io.BytesIO(z.read(excel_name))
        df = pd.read_excel(excel_bytes, sheet_name="PM2.5 Summary")

    return df



url_2019 = (
    "https://data.london.gov.uk/download/"
    "london-atmospheric-emissions-inventory--laei--2019/"
    "17d21cd1-892e-4388-9fea-b48c1b61ee3c/"
    "LAEI-2019-Emissions-Summary-including-Forecast.zip"
)

url_2022 = (
    "https://data.london.gov.uk/download/2lg5g/4ql/"
    "LAEI2022-Emissions-Summary-Excel.zip"
)



pm25_2019_full = load_full_pm25_summary(url_2019)
pm25_2022_full = load_full_pm25_summary(url_2022)


In [ ]:

result_2022 = pm25_2022_full.iloc[[6, 77], 11:]

result_2022

In [ ]:

result_2019 = pm25_2019_full.iloc[[6, 71], 11:]

result_2019

In [ ]:
#choropleth 2022 pollution

import warnings

import geopandas as gpd
import libpysal as lps
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import esda

In [ ]:
geojson_url = "https://hub.arcgis.com/api/v3/datasets/0a92a355a8094e0eb20a7a66cf4ca7cf_10/downloads/data?format=geojson&spatialRefId=4326&where=1%3D1"
output_filename = "london_boroughs.geojson"

In [ ]:
import requests

# Download the GeoJSON file
response = requests.get(geojson_url)
response.raise_for_status()  # Raise an exception for bad status codes

with open(output_filename, 'wb') as f:
    f.write(response.content)

gdf = gpd.read_file(output_filename)

In [ ]:
np.random.seed(12345)
import esda

In [ ]:
pm25_2022_borough = result_2022.T.reset_index()
pm25_2022_borough.columns = pm25_2022_borough.iloc[0]
pm25_2022_borough = pm25_2022_borough[1:]
pm25_2022_borough = pm25_2022_borough.rename(columns={'Row Labels': 'Borough', 'Grand Total': 'PM2.5_2022'})
pm25_2022_borough = pm25_2022_borough[~pm25_2022_borough['Borough'].isin(['Row Labels', 'Non-GLA', 'City of London', 'Grand Total'])]
pm25_2022_borough['PM2.5_2022'] = pd.to_numeric(pm25_2022_borough['PM2.5_2022'])
pm25_2022_borough.head()

In [ ]:
#Standardise borough names: inspect unique values in the 'Borough' column of the pm25_2022_borough DataFrame
pm25_2022_borough['Borough'].unique()

In [ ]:
#same for 'BOROUGH' column of the gdf DataFrame to compare and identify any inconsistencies
gdf['BOROUGH'].unique()

In [ ]:
#make necessary string transformations to the 'Borough' column of pm25_2022_borough to standardize them and ensure consistency with the gdf DataFrame for a successful merge
pm25_2022_borough['Borough'] = pm25_2022_borough['Borough'].replace({
    'Barking and Dagenham': 'Barking & Dagenham',
    'Hammersmith and Fulham': 'Hammersmith & Fulham',
    'Kensington and Chelsea': 'Kensington & Chelsea',
    'Kingston': 'Kingston upon Thames',
    'Richmond': 'Richmond upon Thames'
})

print("Unique borough names in pm25_2022_borough after standardization:")
print(pm25_2022_borough['Borough'].unique())

In [ ]:
#Double check for differences between borough names in pm25_2022_borough and gdf after standardization (City of London can be excluded)
diff_pm25_not_in_gdf = set(pm25_2022_borough['Borough'].unique()) - set(gdf['BOROUGH'].unique())
diff_gdf_not_in_pm25 = set(gdf['BOROUGH'].unique()) - set(pm25_2022_borough['Borough'].unique())

print("Borough names in pm25_2022_borough but not in gdf:", diff_pm25_not_in_gdf)
print("Borough names in gdf but not in pm25_2022_borough:", diff_gdf_not_in_pm25)

In [ ]:
#Merge 2022 PM2.5 data with geographical data: left merge between gdf and pm25_2022_borough using the standardized borough names as keys, drop redundant 'Borough' column
gdf_merged_2022 = gdf.merge(pm25_2022_borough, left_on='BOROUGH', right_on='Borough', how='left')
gdf_merged_2022 = gdf_merged_2022.drop(columns=['Borough'])
gdf_merged_2022.head()

In [ ]:
#Check for any missing values in 'PM2.5_2022' column of the gdf_merged_2022 DataFrame
gdf_merged_2022['PM2.5_2022'].isnull().sum()

In [ ]:
#One missing value indicated in the 'PM2.5_2022' column: display rows in gdf_merged_2022 where this column is null to identify the specific borough(s) that are missing PM2.5 data
gdf_merged_2022[gdf_merged_2022['PM2.5_2022'].isnull()]

In [ ]:
#Since 'City of London' is the only borough with a missing PM2.5 value and was intentionally excluded in the initial data preparation so fill its 'PM2.5_2022' value with 0. This approach aligns with the treatment of the 2019 data and is reasonable given the negligible emissions of the City of London in this context.
gdf_merged_2022['PM2.5_2022'] = gdf_merged_2022['PM2.5_2022'].fillna(0)
gdf_merged_2022[gdf_merged_2022['BOROUGH'] == 'City of London']

In [ ]:
#Double check for null values after filling the 'City of London' entry
gdf_merged_2022['PM2.5_2022'].isnull().sum()

In [ ]:
#Spatial join: use gpd.sjoin with the specified DataFrames, join type, predicate, and suffixes for overlapping columns
sj_gdf = gpd.sjoin(
    gdf,
    gdf_merged_2022,
    how="inner",
    predicate="intersects",
    lsuffix="left",
    rsuffix="right"
)
sj_gdf.head()

In [ ]:
#Calculate mean PM2.5 emissions for each borough
mean_pm25_gb = sj_gdf.groupby('BOROUGH_left')['PM2.5_2022'].mean()
mean_pm25_gb.name = 'PM2.5_2022_Mean'
mean_pm25_gb

In [ ]:
#Merge calculated mean PM2.5 emissions for each borough (mean_pm25_gb) back into the gdf GeoDataFrame
gdf = gdf.merge(mean_pm25_gb, left_on='BOROUGH', right_on='BOROUGH_left', how='left')
gdf.head()

In [ ]:
#Create a choropleth map using the gdf DataFrame, with instructions for plot size, column mapping, color scheme, legend, axis removal, and title, to match 2019 visualisation
import matplotlib.pyplot as plt
import mapclassify as mc

fig, ax = plt.subplots(1, figsize=(12, 10), subplot_kw={'aspect': 'equal'})
gdf.plot(column='PM2.5_2022_Mean', scheme='Quantiles', k=5, cmap='OrRd', legend=True, ax=ax)
ax.set_axis_off()
plt.title('2022 PM2.5 Pollution Across London Boroughs')
plt.show()

In [ ]:
#moran I 2022 pollution

In [ ]:
df = gdf_merged_2022 # Use gdf_merged which contains the PM2.5 data
wq = lps.weights.Queen.from_dataframe(df, use_index=False, silence_warnings=True)
wq.transform = "r"

In [ ]:
y = df["PM2.5_2022"]
ylag = lps.weights.lag_spatial(wq, y)

In [ ]:
ylag

In [ ]:
import mapclassify as mc

ylagq5 = mc.Quantiles(ylag, k=5)

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
df.assign(cl=ylagq5.yb).plot(
    column="cl",
    categorical=True,
    k=5,
    cmap="GnBu",
    linewidth=0.1,
    ax=ax,
    edgecolor="white",
    legend=True,
)
ax.set_axis_off()
plt.title("Spatial Lag Median PM2.5 (Quintiles)")

plt.show()

In [ ]:
wq.transform = "r"
lag_pm25 = lps.weights.lag_spatial(wq, df["PM2.5_2022"]) # Use the correct column name for PM2.5

In [ ]:
pm25 = df["PM2.5_2022"]
b, a = np.polyfit(pm25, lag_pm25, 1)
f, ax = plt.subplots(1, figsize=(9, 9))

plt.plot(pm25, lag_pm25, ".", color="firebrick")

# dashed vert at mean of the price
plt.vlines(pm25.mean(), lag_pm25.min(), lag_pm25.max(), linestyle="--")
# dashed horizontal at mean of lagged price
plt.hlines(lag_pm25.mean(), pm25.min(), pm25.max(), linestyle="--")

# red line of best fit using global I as slope
plt.plot(pm25, a + b * pm25, "r")
plt.title("Moran Scatterplot for 2022 PM2.5 Pollution Across London Boroughs")
plt.ylabel("Spatial Lag of PM2.5")
plt.xlabel("PM2.5")
plt.show()

In [ ]:
#Global Moran's I (slope of best fit line in red)
print(b)

In [ ]:
li = esda.moran.Moran_Local(y, wq)

In [ ]:
#visualise clusters with LISA map

import splot.esda
import matplotlib.pyplot as plt

f, ax = plt.subplots(1, figsize=(9, 9))
splot.esda.lisa_cluster(li, df, p=0.05, ax=ax)
ax.set_axis_off()
plt.title("LISA Cluster Map for 2022 PM2.5 Pollution")
plt.show()